In [1]:
import langchain
from langchain.document_loaders import TextLoader
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
import openai

In [2]:
data=""
import PyPDF2
import re

# open the pdf file
object = PyPDF2.PdfReader("The Monk who sold his ferrari.pdf")

# get number of pages
NumPages = len(object.pages)



# extract text and do the search
for i in range(7, NumPages):
    PageObj = object.pages[i]
    print("this is page " + str(i)) 
    Text = PageObj.extract_text()
    data=data+Text

this is page 7
this is page 8
this is page 9
this is page 10
this is page 11
this is page 12
this is page 13
this is page 14
this is page 15
this is page 16
this is page 17
this is page 18
this is page 19
this is page 20
this is page 21
this is page 22
this is page 23
this is page 24
this is page 25
this is page 26
this is page 27
this is page 28
this is page 29
this is page 30
this is page 31
this is page 32
this is page 33
this is page 34
this is page 35
this is page 36
this is page 37
this is page 38
this is page 39
this is page 40
this is page 41
this is page 42
this is page 43
this is page 44
this is page 45
this is page 46
this is page 47
this is page 48
this is page 49
this is page 50
this is page 51
this is page 52
this is page 53
this is page 54
this is page 55
this is page 56
this is page 57
this is page 58
this is page 59
this is page 60
this is page 61
this is page 62
this is page 63
this is page 64
this is page 65
this is page 66
this is page 67
this is page 68
this is pag

In [3]:
import tiktoken
import redis
import time
from langchain.text_splitter import RecursiveCharacterTextSplitter
# from redisearch_indexer import init, search_redis
import openai
from datetime import datetime
import pandas as pd
from tqdm import tqdm
openai.api_key=''

In [4]:
tokenizer = tiktoken.get_encoding('cl100k_base')
def tiktoken_len(text):

    tokens = tokenizer.encode(

        text,

        disallowed_special=()

    )

    return len(tokens)

In [5]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
    length_function=tiktoken_len,
    separators=["\n\n", "\n", " "]
)

In [6]:
data=str(data)
tiktoken_len(data)

210895

In [7]:
chunks=text_splitter.split_text(data)
print(chunks[0])

attempting to represent Islam’s diversity, I have tried to include material
that tells the story of the major groups within Islam. This means that the
views of the Shı ˆcıˆas well as the Sunnı ˆare included. My choice to do this
is, in part, a corrective. Works of this kind have often been heavily
weighted toward the Sunnı ˆperspective. The reasons for this are
complicated, but it had much to do with the history of how the West
came to learn about Islam and the desire of Western Orientalist writers
to essentialize Islam and not acknowledge the nuances and differences
that they did in Western Christianity. Recognizing complexity in
someone else or in another religious system is an important step toward
understanding that religion as well as one’s own.
This single volume is not intended to be the end and the answer to
questions about Islam, but, rather, a beginning. At the end of the
volume, the reader will ﬁnd a bibliography listing additional English-
language reference works, monograp

In [8]:
len(chunks)

481

In [9]:
## Creating indexes from openai
sk=[]
for embed in tqdm(chunks):
    sk.append(openai.Embedding.create(input = embed, engine='text-embedding-ada-002')['data'][0]['embedding'])
    


100%|██████████| 481/481 [01:07<00:00,  7.08it/s]


In [10]:
data_embeded={"samename":chunks,"openai_embeddings":sk}
import json
out_file = open("abcd.json", "w")  
json.dump(data_embeded, out_file)  
out_file.close()


In [1]:
# import os
# os.chdir("../../")
# os.getcwd()
# # os.system("rm -rf /home/AIAdmin1/Alim_bot/chroma")

'/home/AIAdmin1/Alim_bot'

In [13]:
openai.api_key='sk-proj-9oFspACwXLo6tjgsobG7T3BlbkFJIXBM10NHcdXgdU5F9nQH'
import os
os.environ["OPENAI_API_KEY"] = 'sk-proj-9oFspACwXLo6tjgsobG7T3BlbkFJIXBM10NHcdXgdU5F9nQH'

In [14]:
from langchain.embeddings import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()
embedding_function = embeddings


In [5]:
text = "This is a test document."
query_result = embeddings.embed_query(text)

In [6]:
collection_name = [
    "q/a",
    "hadid",
    "khattab",
    "article"
]

In [15]:
import json 

In [16]:
def create_chroma_index(json_file_path = '', collection_name = '', index_key = None ):
    import chromadb

    f=open(json_file_path)
    data_hadith=json.load(f)
    f.close()
    json_data = data_hadith
    docs = []
    embedding_data = []


    if index_key == None :
        index_key = collection_name


    for index in range(len(data_hadith[index_key])):
        docs.append(data_hadith[index_key][index])
        embedding_data.append(data_hadith["openai_embeddings"][index])
    persistent_client = chromadb.PersistentClient()
    collection = persistent_client.create_collection(collection_name,
                                                    metadata={"hnsw:space": "l2"})
    collection.add( ids = [str(i) for i in range(len(docs))], documents=docs, embeddings=embedding_data)
    print(f"created {collection_name}")
    langchain_chroma = Chroma(
        client=persistent_client,
        collection_name=collection_name,
        embedding_function=embedding_function,
    )
    return langchain_chroma

In [17]:
create_chroma_index(json_file_path='name.json',collection_name = 'name' )

created Quran_English


In [9]:
# create_chroma_index(json_file_path='/home/AIAdmin1/Alim_bot/data/Islamicity_QA/Islamicity_QA_1.json',collection_name = 'Islamicity_QA_1', index_key= "Islamicity_QA"  )


# create_chroma_index(json_file_path='/home/AIAdmin1/Alim_bot/data/Islamicity_QA/Islamicity_QA_2.json',collection_name = 'Islamicity_QA_2', index_key= "Islamicity_QA"  )
# create_chroma_index(json_file_path='/home/AIAdmin1/Alim_bot/data/Islamicity_QA/Islamicity_QA_3.json',collection_name = 'Islamicity_QA_3', index_key= "Islamicity_QA"  )
# create_chroma_index(json_file_path='/home/AIAdmin1/Alim_bot/data/Islamicity_QA/Islamicity_QA_4.json',collection_name = 'Islamicity_QA_4', index_key= "Islamicity_QA"  )
# create_chroma_index(json_file_path='/home/AIAdmin1/Alim_bot/data/Islamicity_hadiths_phase3.json',collection_name = 'Islamicity_hadiths' )
# create_chroma_index(json_file_path='/home/AIAdmin1/Alim_bot/data/Islamicity_khattab_phase3.json',collection_name = 'Islamicity_khattab' )
# create_chroma_index(json_file_path='/home/AIAdmin1/Alim_bot/data/Islamicity_Articles.json',collection_name = 'Islamicity_Articles' )



In [2]:
import os
os.environ["OPENAI_API_KEY"] = 'sk-proj-9oFspACwXLo6tjgsobG7T3BlbkFJIXBM10NHcdXgdU5F9nQH'

def similarity_search(query= "", collection_name = 'Islamicity_khattab', vector = None ):
    from langchain.embeddings import OpenAIEmbeddings
    embeddings = OpenAIEmbeddings()
    embedding_function = embeddings
    from langchain.vectorstores import Chroma
    import chromadb
    index_key = None
    persistent_client = chromadb.PersistentClient()
    # collection = persistent_client.get_collection(collection_name)
    langchain_chroma = Chroma(
    client=persistent_client,
    collection_name=collection_name,
    embedding_function=embedding_function,
    )
    if vector:
        return collection_name, langchain_chroma.similarity_search_by_vector(vector, k = 1)
    return collection_name, langchain_chroma.similarity_search_with_score(query, k = 1)




In [3]:
all_index = [
    "Islamicity_QA_1",
    "Islamicity_QA_2",
    "Islamicity_QA_3",
    "Islamicity_QA_4"
]
checker={}
for i in all_index:
    val=similarity_search("Heyy How are you", i)
    checker[val[1][0][0].page_content]=val[1][0][1]

In [5]:
max(zip(checker.values(), checker.keys()))[1]


'show it off. I\\n", \'Answer:\\n\', \'\\n\', \'All perfect praise be to Allaah, The Lord of the Worlds. I testify that there is none worthy of worship except Allaah, and that Muhammad (saw) is His slave and Messenger. First of all, we ask Allaah to facilitate your affairs, greatly reward you, and increase your piety, righteousness and enable you to perform good deeds.We advise you to be patient about the disease which you have, as there is much good and reward in being patient. The Prophet (saw) said: "How wonderful is the matter of a believer, there is good in all his affairs, and this is only for the believer. If something good happens to him and he is grateful, then this is good for him, and if a calamity befalls him and he is patient, then this is also good for him." [Muslim]. For more benefit, please refer to Fatwa 83577.We also advise you to sincerely turn to Allaah, have trust in Him and earnestly supplicate Him for achieving a benefit or repelling harm. Indeed, you have a good

In [6]:
checker


{'shahzad,mera masla yeeh hai kay kuch 6months phele mere yaha aulad huwee kuch arsa guzarnay kay bad docots nay kaha iss kee ankh mai motia hai hum nay motia kaa ilaj karwaya ab kuch bhetar hai aur docotrs nay ummed dilai hai kay sahee ho jai gaa ahista ahista ab koi yeeh nahee bata sakta kay yeeh kis tarha huwa asal mai taqreeban 28years phele mere walid kay yaha aulad huwee matlab mere behan joo kay motie waqaira kaa kay marz mai thee uss nay mai aur bhee kafee kisam kee bimaria thee mai ap say yeeh chata hoon kay mujee koi iss kisam kaa wird ya duwa batai jis say mere yeeh aulad sahee normal hoo jai aur agee honee walee aulad mai bhee nomal hee hoon mai aap kaa shukar guzar rahoon gaa duwao kaa talabgar shahzad\\n\', \'Answer: As-Salamu `alaykum wa Rahmatullahi wa Barakatuh. In the Name of Allah, Most Gracious, Most Merciful. All praise and thanks are due to Allah, and peace and blessings be upon His Messenger. We commend your keenness on getting your self well-acquainted with Isla